In [2]:
# Install folktables
# !pip install folktables pandas numpy

import pandas as pd
import numpy as np
from folktables import ACSDataSource

# Define the latest available year
latest_year = 2019

# List of all U.S. states (excluding territories)
states_list = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA",
    "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK",
    "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

# Define potential features and target for socioeconomic status prediction
all_features = [
    "AGEP",  # Age
    "SEX",  # Gender
    "RAC1P",  # Race
    "SCHL",  # Education Level
    "MAR",  # Marital Status
    "OCCP",  # Occupation Code
    "WKHP",  # Work Hours per Week
    "COW",  # Class of Worker (Private, Self-employed, etc.)
    "POBP",  # Place of Birth
    "RELP",  # Relationship (Head of Household, Child, etc.)
]

target = "PINCP"  # Personal Income

# Process data in chunks of 5 states at a time
batch_size = 5
for i in range(0, len(states_list), batch_size):
    batch_states = states_list[i:i + batch_size]

    # Load data from ACS survey for the batch of states
    data_source = ACSDataSource(survey_year=latest_year, horizon="1-Year", survey="person")
    acs_data = data_source.get_data(states=batch_states, download=True)

    # Print available columns (debugging step)
    print(f"Available columns for {batch_states}:")
    print(list(acs_data.columns))

    # Filter features that are actually present in the dataset
    available_features = [col for col in all_features if col in acs_data.columns]

    # Convert to Pandas DataFrame
    df = acs_data[available_features + [target]].dropna()  # Drop missing values

    # Convert target into binary classification: Income > $50K = 1, else 0
    df["Income_Above_50K"] = (df["PINCP"] > 50000).astype(int)

    # Drop original income column
    df = df.drop(columns=["PINCP"])

    # Save the chunk as a CSV file
    file_name = f"census_data_{'_'.join(batch_states)}.csv"
    df.to_csv(file_name, index=False)

    print(f"✅ Processed and saved: {file_name} | Shape: {df.shape}")


Available columns for ['AL', 'AK', 'AZ', 'AR', 'CA']:
['RT', 'SERIALNO', 'DIVISION', 'SPORDER', 'PUMA', 'REGION', 'ST', 'ADJINC', 'PWGTP', 'AGEP', 'CIT', 'CITWP', 'COW', 'DDRS', 'DEAR', 'DEYE', 'DOUT', 'DPHY', 'DRAT', 'DRATX', 'DREM', 'ENG', 'FER', 'GCL', 'GCM', 'GCR', 'HIMRKS', 'HINS1', 'HINS2', 'HINS3', 'HINS4', 'HINS5', 'HINS6', 'HINS7', 'INTP', 'JWMNP', 'JWRIP', 'JWTRNS', 'LANX', 'MAR', 'MARHD', 'MARHM', 'MARHT', 'MARHW', 'MARHYP', 'MIG', 'MIL', 'MLPA', 'MLPB', 'MLPCD', 'MLPE', 'MLPFG', 'MLPH', 'MLPI', 'MLPJ', 'MLPK', 'NWAB', 'NWAV', 'NWLA', 'NWLK', 'NWRE', 'OIP', 'PAP', 'RELSHIPP', 'RETP', 'SCH', 'SCHG', 'SCHL', 'SEMP', 'SEX', 'SSIP', 'SSP', 'WAGP', 'WKHP', 'WKL', 'WKWN', 'WRK', 'YOEP', 'ANC', 'ANC1P', 'ANC2P', 'DECADE', 'DIS', 'DRIVESP', 'ESP', 'ESR', 'FOD1P', 'FOD2P', 'HICOV', 'HISP', 'INDP', 'JWAP', 'JWDP', 'LANP', 'MIGPUMA', 'MIGSP', 'MSP', 'NAICSP', 'NATIVITY', 'NOP', 'OC', 'OCCP', 'PAOC', 'PERNP', 'PINCP', 'POBP', 'POVPIP', 'POWPUMA', 'POWSP', 'PRIVCOV', 'PUBCOV', 'QTRBIR', 

In [3]:
import os
import pandas as pd

# List all CSV files in the Colab working directory
csv_files = [file for file in os.listdir() if file.startswith("census_data_") and file.endswith(".csv")]

# Load and concatenate all CSV files into a single DataFrame
df_final = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)

# Save the merged dataset
df_final.to_csv("final_census_dataset.csv", index=False)

print(f"✅ Final dataset saved as 'final_census_dataset.csv' | Shape: {df_final.shape}")


✅ Final dataset saved as 'final_census_dataset.csv' | Shape: (1682005, 10)


In [4]:
import os

# List all files in the current directory
files = os.listdir()
print(files)


['.config', 'final_census_dataset.csv', 'census_data_VA_WA_WV_WI_WY.csv', 'census_data_OK_OR_PA_RI_SC.csv', 'census_data_MT_NE_NV_NH_NJ.csv', 'census_data_KS_KY_LA_ME_MD.csv', 'census_data_NM_NY_NC_ND_OH.csv', 'census_data_AL_AK_AZ_AR_CA.csv', 'census_data_HI_ID_IL_IN_IA.csv', 'census_data_CO_CT_DE_FL_GA.csv', 'census_data_SD_TN_TX_UT_VT.csv', 'data', 'census_data_MA_MI_MN_MS_MO.csv', 'sample_data']


In [6]:
df_final = pd.read_csv("final_census_dataset.csv")

# Display first few rows
print(df_final.head())

   AGEP  SEX  RAC1P  SCHL  MAR    OCCP  WKHP  COW  POBP  Income_Above_50K
0    22    1      1  20.0    5  1555.0  40.0  5.0    17                 0
1    18    2      1  19.0    5  4640.0  20.0  4.0    51                 0
2    19    2      2  19.0    5  2350.0  12.0  1.0    24                 0
3    21    2      1  19.0    5   310.0  10.0  1.0    36                 0
4    21    1      1  18.0    5  3930.0  20.0  1.0    12                 0


In [7]:
print(df_final["Income_Above_50K"].value_counts(normalize=True))

Income_Above_50K
0    0.607511
1    0.392489
Name: proportion, dtype: float64


In [8]:
print(df_final.isnull().sum())

AGEP                0
SEX                 0
RAC1P               0
SCHL                0
MAR                 0
OCCP                0
WKHP                0
COW                 0
POBP                0
Income_Above_50K    0
dtype: int64


In [10]:
print(df_final.dtypes)

AGEP                  int64
SEX                   int64
RAC1P                 int64
SCHL                float64
MAR                   int64
OCCP                float64
WKHP                float64
COW                 float64
POBP                  int64
Income_Above_50K      int64
dtype: object
